<a href="https://colab.research.google.com/github/Hima409/Sentiment-Analysis/blob/main/MYSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv("IMDB1.csv")
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['sentiment']=le.fit_transform(df['sentiment'])
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#text Preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def clean(text):
  text=re.sub(r'[^\w\s]','',text)
  text=text.lower()
  words=text.split()
  words=[word for word in words if word not in stopwords.words('english')]
  text=' '.join(words)
  return text

df['review']=df['review'].apply(clean)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [ ]:
#train test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Example list of reviews (500 rows)
texts = df['review'].values  # Assuming you have a 'review' column in your DataFrame

# Initialize Tokenizer without any limit on num_words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Get the size of the vocabulary
vocab_size = len(tokenizer.word_index)  # Total unique words in your dataset
print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 33106


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

t=Tokenizer(num_words=10000)
t.fit_on_texts(x_train)

x_train_seq=t.texts_to_sequences(x_train)
x_test_seq=t.texts_to_sequences(x_test)

max_len=500
x_train_pad=pad_sequences(x_train_seq,maxlen=max_len)
x_test_pad=pad_sequences(x_test_seq,maxlen=max_len)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout

model=Sequential()
model.add(Embedding(input_dim=10000,output_dim=128,input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.fit(x_train_pad,y_train,validation_data=(x_test_pad,y_test),epochs=5,batch_size=64)


Epoch 1/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 26s 830ms/step - accuracy: 0.5625 - loss: 0.6899 - val_accuracy: 0.7370 - val_loss: 0.6257
Epoch 2/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 731ms/step - accuracy: 0.7800 - loss: 0.5602 - val_accuracy: 0.8087 - val_loss: 0.4796
Epoch 3/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 701ms/step - accuracy: 0.9375 - loss: 0.2388 - val_accuracy: 0.8261 - val_loss: 0.4265
Epoch 4/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 805ms/step - accuracy: 0.9828 - loss: 0.0743 - val_accuracy: 0.8065 - val_loss: 0.6518
Epoch 5/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 803ms/step - accuracy: 0.9895 - loss: 0.0417 - val_accuracy: 0.8174 - val_loss: 0.6136


In [ ]:
model.save('sentiment_analysis_model.h5')

In [ ]:
from google.colab import files
files.download('sentiment_analysis_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
loss,accuracy=model.evaluate(x_test_pad,y_test)
print(f"{accuracy*100:.2f}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.7777 - loss: 0.7358
81.74%


In [ ]:
new_review=""
new_review_clean=clean(new_review)I’m not sure if I loved it or hated it, but I definitely couldn’t stop watching. It was like a chaotic mix of brilliance and nonsense that somehow worked... or maybe it didn’t? Either way, I’ll probably watch it again
new_review_seq=t.texts_to_sequences([new_review_clean])
new_review_pad=pad_sequences(new_review_seq,maxlen=max_len)

prediction=model.predict(new_review_pad)
print(f'Prediction is: {"positive" if prediction>0.5 else "negative"}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Prediction is: positive


In [ ]:
# Save the trained model
model.save("sentiment_model.h5")



In [ ]:
import pickle

# Save the tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(t, f)


In [ ]:
from google.colab import files

# Download the saved model
files.download("sentiment_model.h5")

# Download the tokenizer
files.download("tokenizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>